In [1]:
import joblib,re,os
import numpy as np
import pandas as pd

# read title 

put `Amazon_RawData/titles.txt` in dataframe with product id

In [2]:
with open('./Amazon_RawData/titles.txt','r',encoding = "ISO-8859-1") as f:
    lines = f.read().splitlines()

In [3]:
data = []
invalid = []
pattern = re.compile("[^A-Z0-9]+")
for i,line in enumerate(lines):
    if len(line)<11 or line[10]!=' ':
        invalid.append(i)
        continue
    id = line[:10]
    if pattern.search(id):
        invalid.append(i)
        continue
    else:
        data.append({'id':id,'title':line[11:]})
titles = pd.DataFrame.from_dict(data)

In [4]:
# make sure id is unique
titles.id.is_unique

True

In [21]:
titles.describe()

,id,title
count,1720286,1720286
unique,1720286,1495904
top,B000KNDA6I,
freq,1,1270


# read description

In [6]:
with open('./Amazon_RawData/descriptions.txt','r',encoding = "ISO-8859-1") as f:
    lines = f.read().splitlines()

In [7]:
data = []
pattern = re.compile("[^A-Z0-9]+")
for i,line in enumerate(lines):
    if not line:
        continue
    if line[:18]=='product/productId:':
        _,id = line.split(' ')
        data.append({'id':id})
    elif line[:20] == 'product/description:':
        _,description = line.split(' ',1)
        data[-1]['description'] = description
    else:
        data[-1]['description'] = data[-1]['description']+line

In [8]:
descriptions = pd.DataFrame.from_dict(data)
descriptions.id.is_unique

True

In [16]:
descriptions.describe()

,description,id
count,1495010,1495010
unique,1204096,1495010
top,All products are BRAND NEW and factory sealed....,B0002DX7EQ
freq,6710,1


In [17]:
1186239+306782

1493021

In [20]:
for id in descriptions.id:
    if pattern.search(id) or len(id)<10:
        print(id)

# merge title and description

In [12]:
df = pd.DataFrame.merge(titles,descriptions,how='outer')

In [15]:
df.describe()

,id,title,description
count,2162032,1720286,1495010
unique,2162032,1495904,1204096
top,0521815304,,All products are BRAND NEW and factory sealed....
freq,1,1270,6710


# extract training id
from `AmazonCat-13K_mappings\AmazonCat-13K_test_map.txt`

In [27]:
traindf = pd.read_csv(
    "AmazonCat-13K_mappings/AmazonCat-13K_train_map.txt",
    sep=r'->',
    header=None,
    names=['id','title_mappings'],
)
traindf.id.is_unique

/home/angela/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [28]:
testdf = pd.read_csv(
    "AmazonCat-13K_mappings/AmazonCat-13K_test_map.txt",
    sep=r'->',
    header=None,
    names=['id','title_mappings'],
)
testdf.id.is_unique

/home/angela/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


True

In [36]:
# no duplicated ids
trainid = set(traindf.id.to_list())
testid = set(testdf.id.to_list())
testid&trainid

set()

In [41]:
# test train id are subsets of df
(testid.union(trainid)).issubset(set(df.id.to_list()))

True

# tag